# DicomNiftiConversion.py
Due to some difficulties reading the files in the database, i had to make some changes on the module that reads the data "DicomNiftiConversion.py.
The difficulties met are : 
- SimpleITK does not read philips PET files correctly
- When the manufacturer=='Philips' and 'Units'!=BQML : the suv factor cannot be calculated, it can be retreived at a specific tag.
- How to locate the corrected files in the data base?
- How to locate which corrected Series to use?
- Slice Location : some dicom series (philips) have no slice location tag
- Rescale : rescale slope/intercept give wrong suv values


## Issue 1 : SimpleITK does not read philips PET files correctly : 
### Solution : 
Use of the module dicom2nifti to convert PET series into nifti files, then extracting data with nibabel to multiply by the suv_factor.
Note that nibabel and sitk extract the data with a subtle difference, the image has to be flipped following the 1st dimension to be the way it is supposed to. 

In [ ]:
import dicom2nifti
import dicom2nifti.settings as settings
import nibabel as nib

def convert_dicom_to_nifti(dicom_folder,namae):
    # Convert the DICOM folder to NIfTI
    settings. disable_validate_slice_increment() #ignore slice increment discrepancies 
    dicom2nifti.dicom_series_to_nifti(dicom_folder, namae, reorient_nifti=True)

convert_dicom_to_nifti(seriesDir,'temp.nii')
image_float = nib.load('temp.nii').get_fdata()
image_float = image_float*suv_factor
image_float= np.flip(image_float,1) 
#This mirrors dimension 1 of the image, this way the data is in the same format as sitk extracted data and 
#the model gets data the way it was trained to
suv_img = nib.Nifti1Image(image_float, nib.load('temp.nii').affine, nib.load('temp.nii').header)
nib.save(suv_img, os.path.join(savePath, f'{traits["Patient ID"]}_{traits["Modality"]}_{traits["Study Date"]}.nii.gz'))
os.remove('temp.nii')
return os.path.join(savePath, f'{traits["Patient ID"]}_{traits["Modality"]}_{traits["Study Date"]}.nii.gz')

## Issue 2 : When the manufacturer=='Philips' and 'Units'!=BQML : 
the suv factor cannot be calculated, it can be retreived at a specific tag.

In [ ]:
def bqml_to_suv(dcm_file: pydicom.FileDataset) -> float:
    '''
    Calculates the conversion factor from Bq/mL to SUV bw [g/mL] using 
    the dicom header information in one of the images from a dicom series
    '''
    # TODO: You can access these attributes in a more user friendly way rather
    # than using the codes...change this at some point
    manufacturer = dcm_file[0x00080070].value.lower()
    units = dcm_file[0x00541001].value.lower()
    unitIsNotBqml = "bqml" not in units
    if "philips" in manufacturer and unitIsNotBqml:
        print("Philips")
        suv_factor = float(dcm_file[0x70531000].value)
    else:
        #...
        #Code to calculate the suv for non philips and non bqml series
    Rescale_Slope= dcm_file[0x0028,0x1053].value
    Rescale_Intercept=dcm_file[0x0028,0x1052].value

    return (suv_factor, Rescale_Slope, Rescale_Intercept)

## Issue 3 and 4: How to locate the corrected files in the data base?
The first approach was to look up the 'AttenuationCorrectionMethod' tag : if it is empty, then the series is not corrected, else it is. 
However, sometimes, the tag is not empty and has values like 'None' or something signifying that there is no correction. It is hard to interpret these values.
Another method would be to look up the 'CorrectedImage' tag. This tag gives and array of elements 
that indicate which, if any, corrections have been applied to the images in this Series. To identify which PT series is the corrected one we should look for 
the one with the most corrections. 
Sometimes, there are many series with the maximum number of corrections. In this case we give the user the choice.

The function below makes a dictionnary of all the series with maximum number of corrections. The dictionnary lists the series path and associates it with its description for more visibility.

In [ ]:
def list_of_modality_dirs(input_dir):
    # scan the input dir for directories containing DICOM series
    # determine which directory corresponds to which modality
    dir_list = next(os.walk(input_dir))[1]
    pt_dir_list={}
    ct_dir_list={}
    pt_dir=None
    numberOfCorrections=0
    for direc in dir_list:
        file_list = os.listdir(os.path.join(input_dir, direc))
        for file in file_list:
            filename = os.path.join(input_dir, direc, file)
            if filename.endswith(".dcm"):
                ds = pydicom.read_file(filename)
                Patient_ID = ds.PatientID
                if ds.Modality == 'PT':
                    print('Found PET DIR')
                    if numberOfCorrections<len(ds[0x00280051].value):
                        numberOfCorrections = len(ds[0x00280051].value) #prend le dossier pet avec le plus de corrections
                        pt_dir_list={}
                        pt_dir_list[os.path.join(input_dir,direc)]=ds.SeriesDescription if 'SeriesDescription' in ds else ""
                        break
                    elif numberOfCorrections==len(ds[0x00280051].value):
                        pt_dir_list[os.path.join(input_dir,direc)]=ds.SeriesDescription if 'SeriesDescription' in ds else ""
                        break
                elif ds.Modality == 'CT':
                    print('Found CT DIR')
                    ct_dir_list[os.path.join(input_dir,direc)]=ds.SeriesDescription if 'SeriesDescription' in ds else ""
                    break
                else:
                    print('Found dir without PET or CT')
                    break
    if pt_dir_list=={}:
        print("Could not find PET DIR")
        os._exit(0)
    if ct_dir_list=={}:
        print("Could not find CT DIR")
        os._exit(0)
    else:
        if len(pt_dir_list)>1:
            print(f"Multiple corrected PET files found, please enter the path of the correct file : ", )
            for element in (pt_dir_list):
                print(f'{pt_dir_list[element]}'," "*(max([len(pt_dir_list[j]) for j in pt_dir_list])-len(pt_dir_list[element])),f'{element}')
            choice= input("Enter path : ")
            pt_dir=choice
        else:
            for element in pt_dir_list:
                pt_dir=element
        if len(ct_dir_list)>1:
            print(f"Multiple corrected CT files found, please enter the path of the correct file : ", )
            for element in (ct_dir_list):
                print(f'{ct_dir_list[element]}'," "*(max([len(ct_dir_list[j]) for j in ct_dir_list])-len(ct_dir_list[element])),f'{element}')
            ct_dir= input("Enter path : ")
        else:
            for element in ct_dir_list:
                ct_dir=element
        modality_dirs = {'PT':pt_dir, 'CT':ct_dir, 'ID': Patient_ID}
    return modality_dirs

## Issue 5 : Slice Location : some dicom series (philips) have no SliceLocation tag
Most dicom series have some files that lack the SliceLocation tag, TMTV-NET filters them to avoid abnormalities. Often with philips values, entire
series do not have any SliceLocation tag, and we are left with nothing. The user is warned that at least half dicom files have no SliceLocation, and is 
given the choice to either take all of the dicom files or to filter them.

In [ ]:
def read_slices_from_dir(input_dir):
    # read and sort .dcm slices from a directory
    # first, read all dicom files
    dicom_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".dcm"):
                filename_full = os.path.join(root, file)
                ds = pydicom.read_file(filename_full)
                dicom_files.append(ds)
    # second, only choose files that have 'location' attribure, and sort
    slices = []
    skipcount = 0
    # only include dicom files that represent image slices
    for f in dicom_files:
        if skipcount>=len(dicom_files)//2:
            a=input("Warning : More than half files in series have no SliceLocation tag, do you wish to ignore this tag? (y/n)")
            if a.lower()=='y':
                return dicom_files
            elif a.lower()=='n':
                slices = []
                skipcount = 0
                for f in dicom_files:    
                    if hasattr(f, 'SliceLocation'):
                        slices.append(f)
                    else:
                        skipcount += 1
            #print('Skipped {} files'.format(skipcount))
                slices = sorted(slices,key=lambda s: s.SliceLocation)
                return slices
        if hasattr(f, 'SliceLocation'):
            slices.append(f)
        else:
            skipcount += 1
    #print('Skipped {} files'.format(skipcount))
    slices = sorted(slices,key=lambda s: s.SliceLocation)
    return slices


## Issue 6 : rescale slope/intercept give wrong suv values
We have observed that when multiplying by rescale slope and adding rescale intercept, suv values are wrong. Without these the suv values are correct.